In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

In [5]:
ds = pd.read_csv('stud_math.csv')

In [6]:
display(ds.head(10))
ds.info()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,course,mother,2.0,2.0,0.0,yes,no,no,no,yes,-6.0,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,other,mother,1.0,2.0,3.0,yes,no,NaN,no,yes,-6.0,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,home,mother,1.0,3.0,0.0,no,yes,yes,yes,yes,-9.0,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,home,father,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
5,GP,M,16,U,LE3,T,4.0,3.0,services,other,reputation,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,4.0,2.0,5.0,10.0,75.0
6,GP,M,16,NaN,LE3,T,2.0,2.0,other,other,home,mother,1.0,2.0,0.0,no,no,no,no,yes,-6.0,yes,yes,no,4.0,4.0,4.0,3.0,0.0,55.0
7,GP,F,17,U,GT3,A,4.0,4.0,other,teacher,home,mother,2.0,2.0,0.0,yes,yes,no,no,yes,-6.0,yes,no,no,4.0,1.0,4.0,1.0,6.0,30.0
8,GP,M,15,U,LE3,A,3.0,2.0,services,other,home,mother,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,yes,no,NaN,2.0,2.0,1.0,0.0,95.0
9,GP,M,15,U,NaN,NaN,3.0,4.0,other,other,home,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,5.0,1.0,5.0,0.0,75.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               395 non-null    object 
 1   sex                  395 non-null    object 
 2   age                  395 non-null    int64  
 3   address              378 non-null    object 
 4   famsize              368 non-null    object 
 5   Pstatus              350 non-null    object 
 6   Medu                 392 non-null    float64
 7   Fedu                 371 non-null    float64
 8   Mjob                 376 non-null    object 
 9   Fjob                 359 non-null    object 
 10  reason               378 non-null    object 
 11  guardian             364 non-null    object 
 12  traveltime           367 non-null    float64
 13  studytime            388 non-null    float64
 14  failures             373 non-null    float64
 15  schoolsup            386 non-null    obj

In [7]:
original_cols = ds.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'health', 'absences', 'score'],
      dtype='object')

In [24]:
ds.columns = ['school', 'sex', 'age', 'address', 'family_size', 'parents_status', 'mother_education', 'father_education',
       'mother_job', 'father_job', 'school_choice_reason', 'guardian', 'commute_time', 'homework_time',
       'failures', 'school_support', 'family_support', 'paid_tutor', 'activities', 'pre_school',
       'homework_time_granular', 'college_plan', 'internet_access', 'in_relationship', 'family_climate',
       'free_time', 'out_time', 'health', 'absence', 'score']


In [25]:
ds.nunique()

school                     2
sex                        2
age                        8
address                    2
family_size                2
parents_status             2
mother_education           5
father_education           6
mother_job                 5
father_job                 5
school_choice_reason       4
guardian                   3
commute_time               4
homework_time              4
failures                   4
school_support             2
family_support             2
paid_tutor                 2
activities                 2
pre_school                 2
homework_time_granular     4
college_plan               2
internet_access            2
in_relationship            2
family_climate             6
free_time                  5
out_time                   5
health                     5
absence                   36
score                     18
dtype: int64

In [26]:
for c in ds:
    print(f'{c}:\n{ds[c].value_counts()}\n')

school:
GP    349
MS     46
Name: school, dtype: int64

sex:
F    208
M    187
Name: sex, dtype: int64

age:
16    104
17     98
18     82
15     82
19     24
20      3
22      1
21      1
Name: age, dtype: int64

address:
U    295
R     83
Name: address, dtype: int64

family_size:
GT3    261
LE3    107
Name: family_size, dtype: int64

parents_status:
T    314
A     36
Name: parents_status, dtype: int64

mother_education:
4.0    131
2.0    102
3.0     97
1.0     59
0.0      3
Name: mother_education, dtype: int64

father_education:
2.0     106
3.0      96
4.0      88
1.0      78
0.0       2
40.0      1
Name: father_education, dtype: int64

mother_job:
other       133
services     98
at_home      58
teacher      55
health       32
Name: mother_job, dtype: int64

father_job:
other       197
services    102
teacher      29
at_home      16
health       15
Name: father_job, dtype: int64

school_choice_reason:
course        137
home          104
reputation    103
other          34
Name: schoo

family_climate - отрицательные значения (-1)
father_education - ошибочное значение 40
absence - 385.0 выглядит подозрительно, возможно данные больше чем за 1 год, но мы также видим что 111 человек имеет 0 пропусков, что с точки зрения здравого смысла трудно достижимо с увеличеним числа годов.